In [1]:
import pandas as pd
import numpy as np
from math import pi
import scipy.stats as stats
import scipy.sparse as sps

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

from stat_inference_helpers import evaluate_model
from ML_helpers import preprocess_data

from sklearn.metrics import r2_score, explained_variance_score, mean_squared_error, mean_absolute_error

from joblib import dump, load

In [2]:
data_info = pd.read_parquet("./data/data_info.parquet")
data = pd.read_csv("./data/data_train.csv")
results = pd.read_csv("./data/results.csv")
results

,Model,List of used features,RMSE_train,RMSE_valid,MAE_train,MAE_valid,r2_train,r2_valid,expl_variance_train,expl_variance_valid,corr_coef_train,corr_coef_valid,VIFs
0,LinearRegression(),"['log_Area', 'Manhattan_distance', 'District_Z...",225837,271648,106479,102456,0.850789,0.788447,0.851855,0.794030,0.9156,0.924577,"[3.81, 8.34, 3.13, 4.33]"
1,LinearRegression(),"['log_Area', 'Manhattan_distance', 'District_N...",223732,270236,98167,92048,0.869028,0.790640,0.869805,0.796803,0.9142,0.947081,"[6.99, 12.98, 3.82, 2.81, 2.21, 1.88, 2.47, 5.38]"
2,LinearRegression(),"['log_Area', 'Manhattan_distance', 'Region_1',...",177163,271648,98980,102456,0.850789,0.788447,0.851855,0.794030,0.9134,0.924577,"[3.82, 8.77, inf, inf, 3.98]"
3,LinearRegression(),"['log_Area', 'Manhattan_distance', 'Region_1']",220496,272621,101764,101860,0.846804,0.786928,0.847811,0.792099,0.8952,0.920842,"[3.87, 5.5, 1.9]"
4,LinearRegression(),"['log_Area', 'District_Zuid-Oost', 'District_t...",254188,335543,124477,137928,0.782330,0.677223,0.783123,0.687628,0.8582,0.858843,"[2.71, 1.19, 2.52]"
5,LinearRegression(),"['log_Area', 'Manhattan_distance']",205065,269732,101122,101547,0.846360,0.791421,0.847387,0.796526,0.9013,0.921296,"[3.34, 3.34]"


There still might be room for improvement. First, let's start with the last point of the Statistical Inference Summary: **Cut the maximum `Price`**.

But before, let's discover how many observations lay behind the threshold of 1M, 1.5M and 2M:

In [3]:
total_n = data.shape[0]
threaholds = [1000000, 1500000, 2000000]

for threshold in threaholds:
    above_treshold = (data[data['Price']>threshold]).shape[0]
    print(f'There are {round((above_treshold*100/total_n), 2)}% of points with price above {threshold}')


There are 11.26% of points with price above 1000000
There are 4.48% of points with price above 1500000
There are 1.9% of points with price above 2000000


11% is too much to cut off, but it's possible to try with 1.5M and 2M.

**1.** Cut off all datapoints with `Price` > 2M

In [4]:
data7 = data.copy()
data7.drop(data[data['Price']> 2000000].index, inplace=True)
data7

,Price,Area,Price per sqm,log_Price,log_Area,Manhattan_distance,Zip region,Zip num,District_v1,District_v2
0,385000.0,49,7857.142857,12.860999,3.891820,0.051544,0,1094,Oost,the_rest
1,930000.0,123,7560.975610,13.742940,4.812184,0.029115,0,1079,Zuid,Centrum+Zuid
2,500000.0,70,7142.857143,13.122363,4.248495,0.038423,0,1051,West,the_rest
3,400000.0,107,3738.317757,12.899220,4.672829,0.107977,0,1067,Nieuw-West,the_rest
4,475000.0,98,4846.938776,13.071070,4.584967,0.067423,0,1061,Nieuw-West,the_rest
...,...,...,...,...,...,...,...,...,...,...
732,1025000.0,135,7592.592593,13.840203,4.905275,0.029167,0,1077,Zuid,Centrum+Zuid
733,915000.0,88,10397.727273,13.726679,4.477337,0.017201,0,1015,Centrum,Centrum+Zuid
734,690000.0,100,6900.000000,13.444447,4.605170,0.039710,0,1014,West,the_rest
735,450000.0,60,7500.000000,13.017003,4.094345,0.045514,0,1093,Oost,the_rest


Run the last model on new data:

In [5]:
X7 = data7.drop(columns=['Price', 'Area', 'Price per sqm', 'log_Price', 'Zip region', 'Zip num', 'District_v1', 'District_v2'])
y7 = data7['log_Price']
model7 = LinearRegression()

In [6]:
evaluate_model(model7, X7, y7, results, cv = 4)
results

,Model,List of used features,RMSE_train,RMSE_valid,MAE_train,MAE_valid,r2_train,r2_valid,expl_variance_train,expl_variance_valid,corr_coef_train,corr_coef_valid,VIFs
0,LinearRegression(),"['log_Area', 'Manhattan_distance', 'District_Z...",225837,271648,106479,102456,0.850789,0.788447,0.851855,0.794030,0.9156,0.924577,"[3.81, 8.34, 3.13, 4.33]"
1,LinearRegression(),"['log_Area', 'Manhattan_distance', 'District_N...",223732,270236,98167,92048,0.869028,0.790640,0.869805,0.796803,0.9142,0.947081,"[6.99, 12.98, 3.82, 2.81, 2.21, 1.88, 2.47, 5.38]"
2,LinearRegression(),"['log_Area', 'Manhattan_distance', 'Region_1',...",177163,271648,98980,102456,0.850789,0.788447,0.851855,0.794030,0.9134,0.924577,"[3.82, 8.77, inf, inf, 3.98]"
3,LinearRegression(),"['log_Area', 'Manhattan_distance', 'Region_1']",220496,272621,101764,101860,0.846804,0.786928,0.847811,0.792099,0.8952,0.920842,"[3.87, 5.5, 1.9]"
4,LinearRegression(),"['log_Area', 'District_Zuid-Oost', 'District_t...",254188,335543,124477,137928,0.782330,0.677223,0.783123,0.687628,0.8582,0.858843,"[2.71, 1.19, 2.52]"
5,LinearRegression(),"['log_Area', 'Manhattan_distance']",205065,269732,101122,101547,0.846360,0.791421,0.847387,0.796526,0.9013,0.921296,"[3.34, 3.34]"
6,LinearRegression(),"[log_Area, Manhattan_distance]",140345,161243,88788,83922,0.838505,0.706576,0.840612,0.711736,0.9021,0.906434,"[3.37, 3.37]"


Even though the r2 and percentage of explained variance are now lower, both **errors** have been **signiffacantly redused**.

**2.** Cut off all datapoints with `Price` > 1.5M

In [7]:
data8 = data7.copy()
data8.drop(data7[data7['Price']> 1500000].index, inplace=True)
data8.shape

(704, 10)

In [8]:
X8 = data8.drop(columns=['Price', 'Area', 'Price per sqm', 'log_Price', 'Zip region', 'Zip num', 'District_v1', 'District_v2'])
y8 = data8['log_Price']
model8 = LinearRegression()

In [9]:
evaluate_model(model8, X8, y8, results, cv = 4)
results

,Model,List of used features,RMSE_train,RMSE_valid,MAE_train,MAE_valid,r2_train,r2_valid,expl_variance_train,expl_variance_valid,corr_coef_train,corr_coef_valid,VIFs
0,LinearRegression(),"['log_Area', 'Manhattan_distance', 'District_Z...",225837,271648,106479,102456,0.850789,0.788447,0.851855,0.794030,0.9156,0.924577,"[3.81, 8.34, 3.13, 4.33]"
1,LinearRegression(),"['log_Area', 'Manhattan_distance', 'District_N...",223732,270236,98167,92048,0.869028,0.790640,0.869805,0.796803,0.9142,0.947081,"[6.99, 12.98, 3.82, 2.81, 2.21, 1.88, 2.47, 5.38]"
2,LinearRegression(),"['log_Area', 'Manhattan_distance', 'Region_1',...",177163,271648,98980,102456,0.850789,0.788447,0.851855,0.794030,0.9134,0.924577,"[3.82, 8.77, inf, inf, 3.98]"
3,LinearRegression(),"['log_Area', 'Manhattan_distance', 'Region_1']",220496,272621,101764,101860,0.846804,0.786928,0.847811,0.792099,0.8952,0.920842,"[3.87, 5.5, 1.9]"
4,LinearRegression(),"['log_Area', 'District_Zuid-Oost', 'District_t...",254188,335543,124477,137928,0.782330,0.677223,0.783123,0.687628,0.8582,0.858843,"[2.71, 1.19, 2.52]"
5,LinearRegression(),"['log_Area', 'Manhattan_distance']",205065,269732,101122,101547,0.846360,0.791421,0.847387,0.796526,0.9013,0.921296,"[3.34, 3.34]"
6,LinearRegression(),"[log_Area, Manhattan_distance]",140345,161243,88788,83922,0.838505,0.706576,0.840612,0.711736,0.9021,0.906434,"[3.37, 3.37]"
7,LinearRegression(),"[log_Area, Manhattan_distance]",111131,112299,73936,73227,0.828656,0.774253,0.831635,0.774852,0.9039,0.858599,"[3.48, 3.48]"


Even though the diffence between 2 models may seem insignificant on training set, it is significant on validation set. What is even more valuable is the delta of all metrics of the last model between train and validation set. Delta is low, which says the model fits really good. Next we will use cut-off to 1.5M.

Let's check if this model is same good on test set:

In [10]:
data_test = pd.read_csv("./data/data_test.csv").dropna(how = 'any')
data_test.drop(data_test[data_test['Price']> 1500000].index, inplace=True)

# reset indexes
data_test = data_test.reset_index()
data_test = data_test.drop(columns=['index'])
data_test.tail()

,Address,Zip,Price,Area,Room,Lon,Lat
174,"Wieringerwaardstraat 221, Amsterdam",1024 HL,250000.0,50,2,4.959052,52.395800
175,"Winterjanpad 21, Amsterdam",1036 KN,720000.0,138,5,4.902503,52.410538
176,"Paramaribostraat 122 3, Amsterdam",1058 VP,700000.0,102,6,4.854520,52.362090
177,"Leeuwendalersweg 636, Amsterdam",1061 BJ,465000.0,94,4,4.838028,52.377317
178,"Tenerifestraat 8, Amsterdam",1060 TH,699000.0,130,5,4.788762,52.344183


Data engineering and transformation

In [11]:
data_test['log_Area'] = np.log(data_test['Area'])
data_test['log_Price'] = np.log(data_test['Price'])

center_lat = 52.36277216974363
center_lon = 4.889643667868924

data_test = data_test.assign(
    **{
        "Manhattan_distance": lambda df: np.abs(center_lat - df["Lat"]) + np.abs(center_lon - df["Lon"])
    }
)

data_test = data_test.drop(columns = ['Address', 'Zip', 'Room', 'Lon', 'Lat'])
data_test.sample()

,Price,Area,log_Area,log_Price,Manhattan_distance
162,325000.0,77,4.343805,12.69158,0.090656


In [12]:
X_test = data_test.drop(columns = ['Price', 'log_Price', 'Area'])
y_test = data_test['log_Price']

# as we're going to use the same model we did for cut-off of 1.5M, lets now fit it a make predictions on test data and then evaluate it.
model8.fit(X8, y8)

y_test_pred = model8.predict(X_test)

# back transformation of results
y_test_true = np.exp(y_test)
y_test_pred = np.exp(y_test_pred)

In [13]:
# evaluate results
RMSE = int(mean_squared_error(y_test_true, y_test_pred, squared=False))
MAE = int(mean_absolute_error(y_test_true, y_test_pred))
r2 = r2_score(y_test_true, y_test_pred)
corr = stats.spearmanr(y_test_true, y_test_pred)[0]
explained_variance_train = explained_variance_score(y_test_true, y_test_pred)

# add the above metrics to the results table
new_row = ['LR on test data (<1.5M)', list(X_test.columns), 'N/A', RMSE, 'N/A', MAE, 'N/A', r2, 'N/A', corr, 'N/A', explained_variance_train, 'N/A']
results.loc[len(results)] = new_row
results

,Model,List of used features,RMSE_train,RMSE_valid,MAE_train,MAE_valid,r2_train,r2_valid,expl_variance_train,expl_variance_valid,corr_coef_train,corr_coef_valid,VIFs
0,LinearRegression(),"['log_Area', 'Manhattan_distance', 'District_Z...",225837,271648,106479,102456,0.850789,0.788447,0.851855,0.794030,0.9156,0.924577,"[3.81, 8.34, 3.13, 4.33]"
1,LinearRegression(),"['log_Area', 'Manhattan_distance', 'District_N...",223732,270236,98167,92048,0.869028,0.790640,0.869805,0.796803,0.9142,0.947081,"[6.99, 12.98, 3.82, 2.81, 2.21, 1.88, 2.47, 5.38]"
2,LinearRegression(),"['log_Area', 'Manhattan_distance', 'Region_1',...",177163,271648,98980,102456,0.850789,0.788447,0.851855,0.794030,0.9134,0.924577,"[3.82, 8.77, inf, inf, 3.98]"
3,LinearRegression(),"['log_Area', 'Manhattan_distance', 'Region_1']",220496,272621,101764,101860,0.846804,0.786928,0.847811,0.792099,0.8952,0.920842,"[3.87, 5.5, 1.9]"
4,LinearRegression(),"['log_Area', 'District_Zuid-Oost', 'District_t...",254188,335543,124477,137928,0.78233,0.677223,0.783123,0.687628,0.8582,0.858843,"[2.71, 1.19, 2.52]"
5,LinearRegression(),"['log_Area', 'Manhattan_distance']",205065,269732,101122,101547,0.84636,0.791421,0.847387,0.796526,0.9013,0.921296,"[3.34, 3.34]"
6,LinearRegression(),"[log_Area, Manhattan_distance]",140345,161243,88788,83922,0.838505,0.706576,0.840612,0.711736,0.9021,0.906434,"[3.37, 3.37]"
7,LinearRegression(),"[log_Area, Manhattan_distance]",111131,112299,73936,73227,0.828656,0.774253,0.831635,0.774852,0.9039,0.858599,"[3.48, 3.48]"
8,LR on test data (<1.5M),"[log_Area, Manhattan_distance]",N/A,148388,N/A,96331,N/A,0.740465,N/A,0.854582,N/A,0.746625,N/A


The model underfits, thus the results on the test set are slightly worse. However, the RMSE and MAE are still better than nearly all of validation results from training models before. Let's try to regularize this model.

**Regularized Linear Regression**

Let us now try to use regularized versions of Linear Regression: Lasso, Ridge and ElasticNet Regression. Since these models can carry out feature selection (embedded methods), we can relax the condition of having a feature set with a VIF score > 5 and let the model decide the optimal weights for each feature. We will use the following set of features: 
- log(Area)
- Manhattan_distance
- Zip region
- District_v2

In [14]:
models_alphas = {}

In [15]:
def get_best_alpha(model:str, 
                  data:pd.DataFrame, 
                  numerical_features:list, categorical_features:list, 
                  ordinal_feature: str = '', order_of_categories: list = [], 
                  n_alphas: int = 100, alphas_range: list = [1e-3, 5]
                ):
    
    """Gridseraches the best alpha of selected regularization model for selected dataset

    Args:
        model: str
            Name of regularization models. Supperted models: Lasso, Ridge, ElasticNet
        data: pd.DataFrame
            The dataset, from which you choose needed set of features
        numerical_features: list
            List of features, that have to be preprocessed as numetical
        categorical_features: list 
            List of features, that have to be preprocessed as categorical
        ordinal_feature: str
            Optional. Feature, that have to be preprocessed as ordinal
        order_of_categories: list
            Optional. Here you have to provide the right ascending order of values of the ordinal feature as a list 
        n_alphas: int, optional
            Optional. Number of alpha values you want to check. Defaults to 100.
        range: list, optional
            Optional. List of 2 numbers: the lowest alpha you want to check anf the highest. Defaults to [1e-3, 5].

    Raises:
        ValueError: _description_

    Returns:
        _type_: _description_
    """

    model_type = model.lower()

    if model_type not in ['lasso', 'ridge', 'elasticnet']:
        raise ValueError("Model type not supported")
    
    preprocessed_data = preprocess_data(data, 
                                        data.columns[0], 
                                        numerical_features, 
                                        categorical_features, 
                                        ordinal_feature, order_of_categories)
    if model_type == 'lasso':
        model = Lasso(fit_intercept=True)
    if model_type == 'ridge':
        model = Ridge(fit_intercept=True)
    if model_type == 'elasticnet':
        model = ElasticNet(fit_intercept=True)

    alpha_range = np.linspace(alphas_range[0], alphas_range[-1], n_alphas)
    param_grid = {"alpha": alpha_range}

    gridCV = GridSearchCV(model, param_grid, scoring='r2')
    _ = gridCV.fit(preprocessed_data.iloc[:, 1:], preprocessed_data.iloc[:, 0])

    best_alpha = gridCV.best_params_["alpha"]
    models_alphas[model_type] = [best_alpha]

    return models_alphas
        
        

In [16]:
get_best_alpha('lasso', data, numerical_features=['Area', 'Manhattan_distance'], categorical_features=['District_v2', 'Zip region'])
get_best_alpha('ridge', data, numerical_features=['Area', 'Manhattan_distance'], categorical_features=[])
get_best_alpha('elasticNet', data, numerical_features=['Area', 'Manhattan_distance'], categorical_features=[])
models_alphas


{'lasso': [0.001], 'ridge': [5.0], 'elasticnet': [0.001]}

Let's now run the evaluation funcrion for each model:

In [17]:
data9 = data.copy()
data9.drop(data9[data9['Price']> 1500000].index, inplace=True)
data9.shape

# because Lasso can eliminate unimportant features, we'll feed it everything
X9 = data9.drop(columns=['Price', 'Area', 'Price per sqm', 'log_Price', 'Zip num', 'District_v1', ])
region = pd.get_dummies(X9['Zip region'], drop_first=True, dtype=int, prefix='Region')
district = pd.get_dummies(X9['District_v2'], drop_first=True, dtype=int, prefix='District')
X9 = pd.concat([X9, district], axis = 1).drop(columns=['Zip region', 'District_v2'])

# for Ridge and ElasticNet we'll use only area and manhattan distance
X10 = data9.drop(columns=['Price', 'Area', 'Price per sqm', 'Zip region','log_Price', 'Zip num', 'District_v1', 'District_v2'])

y9 = data9['log_Price']
model9 = Lasso(alpha=1e-3)
model10 = Ridge(alpha=5)
model11 = ElasticNet(alpha=1e-3)

In [18]:
evaluate_model(model9, X9, y9, results, cv = 4)
evaluate_model(model10, X10, y9, results, cv = 4)
evaluate_model(model11, X10, y9, results, cv = 4)

results

,Model,List of used features,RMSE_train,RMSE_valid,MAE_train,MAE_valid,r2_train,r2_valid,expl_variance_train,expl_variance_valid,corr_coef_train,corr_coef_valid,VIFs
0,LinearRegression(),"['log_Area', 'Manhattan_distance', 'District_Z...",225837,271648,106479,102456,0.850789,0.788447,0.851855,0.794030,0.9156,0.924577,"[3.81, 8.34, 3.13, 4.33]"
1,LinearRegression(),"['log_Area', 'Manhattan_distance', 'District_N...",223732,270236,98167,92048,0.869028,0.790640,0.869805,0.796803,0.9142,0.947081,"[6.99, 12.98, 3.82, 2.81, 2.21, 1.88, 2.47, 5.38]"
2,LinearRegression(),"['log_Area', 'Manhattan_distance', 'Region_1',...",177163,271648,98980,102456,0.850789,0.788447,0.851855,0.794030,0.9134,0.924577,"[3.82, 8.77, inf, inf, 3.98]"
3,LinearRegression(),"['log_Area', 'Manhattan_distance', 'Region_1']",220496,272621,101764,101860,0.846804,0.786928,0.847811,0.792099,0.8952,0.920842,"[3.87, 5.5, 1.9]"
4,LinearRegression(),"['log_Area', 'District_Zuid-Oost', 'District_t...",254188,335543,124477,137928,0.78233,0.677223,0.783123,0.687628,0.8582,0.858843,"[2.71, 1.19, 2.52]"
5,LinearRegression(),"['log_Area', 'Manhattan_distance']",205065,269732,101122,101547,0.84636,0.791421,0.847387,0.796526,0.9013,0.921296,"[3.34, 3.34]"
6,LinearRegression(),"[log_Area, Manhattan_distance]",140345,161243,88788,83922,0.838505,0.706576,0.840612,0.711736,0.9021,0.906434,"[3.37, 3.37]"
7,LinearRegression(),"[log_Area, Manhattan_distance]",111131,112299,73936,73227,0.828656,0.774253,0.831635,0.774852,0.9039,0.858599,"[3.48, 3.48]"
8,LR on test data (<1.5M),"[log_Area, Manhattan_distance]",N/A,148388,N/A,96331,N/A,0.740465,N/A,0.854582,N/A,0.746625,N/A
9,Lasso(alpha=0.001),"[log_Area, Manhattan_distance, District_Zuid-O...",113097,115677,77554,76312,0.82984,0.760466,0.831736,0.761628,0.8929,0.856657,"[5.16, 8.6, 3.31, 4.77]"


The ElasticNet looks the best of 3: difference between training and validation errors is minimal and actually even lower on validation set. Let's check it on test data. We'll use previously designed X and y.

In [19]:
model11.fit(X10, y9)

y_test_pred = model11.predict(X_test)

# back transformation of results
y_test_pred = np.exp(y_test_pred)

# evaluate results
RMSE = int(mean_squared_error(y_test_true, y_test_pred, squared=False))
MAE = int(mean_absolute_error(y_test_true, y_test_pred))
r2 = r2_score(y_test_true, y_test_pred)
corr = stats.spearmanr(y_test_true, y_test_pred)[0]
explained_variance_train = explained_variance_score(y_test_true, y_test_pred)

# add the above metrics to the results table
new_row = [f'{model11} on test data (<1.5M)', list(X_test.columns), 'N/A', RMSE, 'N/A', MAE, 'N/A', r2, 'N/A', corr, 'N/A', explained_variance_train, 'N/A']
results.loc[len(results)] = new_row
results

,Model,List of used features,RMSE_train,RMSE_valid,MAE_train,MAE_valid,r2_train,r2_valid,expl_variance_train,expl_variance_valid,corr_coef_train,corr_coef_valid,VIFs
0,LinearRegression(),"['log_Area', 'Manhattan_distance', 'District_Z...",225837,271648,106479,102456,0.850789,0.788447,0.851855,0.794030,0.9156,0.924577,"[3.81, 8.34, 3.13, 4.33]"
1,LinearRegression(),"['log_Area', 'Manhattan_distance', 'District_N...",223732,270236,98167,92048,0.869028,0.790640,0.869805,0.796803,0.9142,0.947081,"[6.99, 12.98, 3.82, 2.81, 2.21, 1.88, 2.47, 5.38]"
2,LinearRegression(),"['log_Area', 'Manhattan_distance', 'Region_1',...",177163,271648,98980,102456,0.850789,0.788447,0.851855,0.794030,0.9134,0.924577,"[3.82, 8.77, inf, inf, 3.98]"
3,LinearRegression(),"['log_Area', 'Manhattan_distance', 'Region_1']",220496,272621,101764,101860,0.846804,0.786928,0.847811,0.792099,0.8952,0.920842,"[3.87, 5.5, 1.9]"
4,LinearRegression(),"['log_Area', 'District_Zuid-Oost', 'District_t...",254188,335543,124477,137928,0.78233,0.677223,0.783123,0.687628,0.8582,0.858843,"[2.71, 1.19, 2.52]"
5,LinearRegression(),"['log_Area', 'Manhattan_distance']",205065,269732,101122,101547,0.84636,0.791421,0.847387,0.796526,0.9013,0.921296,"[3.34, 3.34]"
6,LinearRegression(),"[log_Area, Manhattan_distance]",140345,161243,88788,83922,0.838505,0.706576,0.840612,0.711736,0.9021,0.906434,"[3.37, 3.37]"
7,LinearRegression(),"[log_Area, Manhattan_distance]",111131,112299,73936,73227,0.828656,0.774253,0.831635,0.774852,0.9039,0.858599,"[3.48, 3.48]"
8,LR on test data (<1.5M),"[log_Area, Manhattan_distance]",N/A,148388,N/A,96331,N/A,0.740465,N/A,0.854582,N/A,0.746625,N/A
9,Lasso(alpha=0.001),"[log_Area, Manhattan_distance, District_Zuid-O...",113097,115677,77554,76312,0.82984,0.760466,0.831736,0.761628,0.8929,0.856657,"[5.16, 8.6, 3.31, 4.77]"


Simple linera regression without regularization on data with prices below 1.5M still has the best performance. We will use it for predictions. 

First, we need to create a full pipeline to preprocess data and then feed to model (also inside the pipeline). Then we'll save it as a `.joblib` file and use in `dash` application to predict new values.

**1. Create a pipeline:**

In [20]:
data.sample()
final_data = data.copy().drop(columns=['Price', 'Area', 'Price per sqm', 'Zip region', 'Zip num', 'District_v1', 'District_v2'])

# save clean data for model training
final_data.to_csv('data/final_data.csv', index = False)


In [22]:
numerical_variables = ['log_Area']

columntransformer = ColumnTransformer(transformers = [
        ('stand scaler', StandardScaler(), numerical_variables)],
        remainder='passthrough')

pipe = make_pipeline(columntransformer, LinearRegression())

dump(pipe, 'LR_Ams_housing.joblib')

['LR_Ams_housing.joblib']